In [1]:
#import anvil.server

#anvil.server.connect("PLPREEFX77QBTZXIMMGXT5S5-LLZOQE4RLL7KOL6G")

import anvil.server
from anvil import Image, XYPanel

anvil.server.connect("N2RJZAZY3RKJ65AUOA5JKOLC-GFGPDCRRTFT46NHQ")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


In [2]:
# Importing all the utilities
from utilities_stats import *

from anvil import URLMedia
import anvil.media
import datetime
from datetime import date

from joblib import Parallel, delayed

#Setting the total number of matchdays in the season
tot_giornate = 35

# setting useful parameters
Teams, Logos, parameters, Results_0, goal_marks = set_par()
giornate = current_matchday()

# filling individual dataframes and conditional display
Results = fill_dataframe_partita(Results_0, giornate, parameters, goal_marks, Teams, Print = False)

# saving cumulative statistical data
pf_med, pf_std, ps_med, ps_std, gf_med, gf_std = cumulative_data(Results, giornate, Print = False)

# Building cumulative dataframe
Total = make_Total_df(Results, giornate, goal_marks)
Tot_per_round = partial_totals(Results, giornate, tot_giornate, goal_marks)
#Total.head(10)


In [3]:
@anvil.server.callable
def current_matchDay():
    giornate = current_matchday()
    return giornate

In [4]:
@anvil.server.callable

def generate_plots():
    global dict_out
    dict_out = {}
    for premio in ['Caduti','Porta Violata','Catenaccio','Panchina Oro','Cartellino Facile']:
        dict_out[premio] = premio_plot(Results, giornate, Teams, Logos, premio)
    fortuna_evo(Results, Teams, Tot_per_round)
    C_factor_logos_2(Total, giornate, Teams, tot_giornate, Logos)

In [40]:
generate_plots()

In [6]:
import json
from pymongo import MongoClient
from pprint import pprint
import pymongo

from datetime import datetime


with open('credential.json','r') as f:
    cred = json.load(f)
    
    
cluster = MongoClient(cred['cred'])
# choosing database
db = cluster["Game"]
# choosing collection
collection = db["Players"]
collection_man = db['Managers']
collection_tr = db['Transfers']

In [7]:
@anvil.server.callable
def man_team_name(owner):
    owner = owner.lower()
    dic = collection_man.find_one({'owner': owner})
    
    return dic['team_name']

In [8]:
@anvil.server.callable

def test_funct(owner, squad):
    owner = owner.lower()
    squad = squad.lower()
    players = []
    roles = []
    loan = []
    value_init = 0
    value_now = 0
    posts = collection.find({'info.current_team.owner': owner,'info.current_team.squad': squad})
    for player in posts:
        value_init += player['info']['current_team']['quotation_initial']
        value_now += player['info']['stats']['Qt_A']
        players.append(player['name'])
        roles.append(player['info']['personal_info']['FC_role'])
        temp = ''
        if player['info']['current_team']['on_loan']:
            temp = 'on loan'
        loan.append(temp)
    return players, roles, loan, value_init, value_now

In [9]:
collection.find_one({'name':'LASAGNA'})['info']['current_team']['previous_team']

'nanni, main'

In [10]:
@anvil.server.callable
def stats_link_up(name):
    dic = collection.find_one({'name': name})
    name_url = dic['name']
    name_url = name_url.replace(' ','-')
    name_url = name_url.replace('.','')
    stats_link = 'https://www.fantacalcio.it/squadre/'+dic['info']['personal_info']['team_real']+'/'+name_url+'/'+str(dic['_id'])
    return stats_link

In [11]:
@anvil.server.callable

def rose_funct(owner, squad):
    owner = owner.lower()
    squad = squad.lower()
    players = []
    
    value_init = 0
    value_now = 0
    posts = collection.find({'info.current_team.owner': owner,'info.current_team.squad': squad})
    
    mean_age = 0
    tot_cost = 0
    
    for player in posts:
        name_url = player['name']
        name_url = name_url.replace(' ','-')
        name_url = name_url.replace('.','')
        dag = ''
        if player['info']['personal_info']['team_real'] is None:
            stats_link = ''
            dag = '\u2020'
        else:
            stats_link = 'https://www.fantacalcio.it/squadre/'+player['info']['personal_info']['team_real']+'/'+name_url+'/'+str(player['_id'])
        temp = ''
        if player['info']['current_team']['on_loan']:
            temp = '*'
        age = int(np.floor((datetime.today()-datetime.strptime(player['info']['personal_info']['birthdate'], "%d/%m/%Y")).days/365.4))
        players.append({'role': player['info']['personal_info']['FC_role']
                        , 'name': player['name']
                        , 'age': age
                        , 'quotation': player['info']['stats']['Qt_A']
                        , 'quotation_initial': player['info']['contract']['quotation_initial']
                        , 'loan': temp+dag,
                       'link': stats_link,
                       'owner':owner,
                        'cost':player['info']['contract']['cost']})
        value_init += player['info']['current_team']['quotation_initial']
        value_now += player['info']['stats']['Qt_A']
        mean_age += age
        tot_cost += player['info']['contract']['cost']
    
    
    posts = collection.find({'info.contract.owner': owner,'info.current_team.on_loan': True})
    for player in posts:
        name_url = player['name']
        name_url = name_url.replace(' ','-')
        name_url = name_url.replace('.','')
        dag = ''
        if player['info']['personal_info']['team_real'] is None:
            stats_link = ''
            dag = '\u2020'
        else:
            stats_link = 'https://www.fantacalcio.it/squadre/'+player['info']['personal_info']['team_real']+'/'+name_url+'/'+str(player['_id'])
        
        temp = '**'
        if squad not in player['info']['current_team']['previous_team']:
            continue
        age = int(np.floor((datetime.today()-datetime.strptime(player['info']['personal_info']['birthdate'], "%d/%m/%Y")).days/365.4))
        players.append({'role': player['info']['personal_info']['FC_role']
                        , 'name': player['name']
                        , 'age': age
                        , 'quotation': player['info']['stats']['Qt_A']
                        , 'quotation_initial': player['info']['contract']['quotation_initial']
                        , 'loan': temp+dag,
                       'link': stats_link,
                       'owner':owner,
                        'cost':player['info']['current_team']['loan_info']['cost']})
        value_init += player['info']['current_team']['quotation_initial']
        value_now += player['info']['stats']['Qt_A']
        mean_age += age
        tot_cost += player['info']['current_team']['loan_info']['cost']
    
    mean_age = mean_age/len(players)
    return players, value_init, value_now, round(mean_age, 1), man_team_name(owner), tot_cost

In [12]:
@anvil.server.callable
def rose_funct_all(owner):
    return rose_funct(owner, 'main'), rose_funct(owner, 'primavera')

In [13]:
@anvil.server.callable

def man_data(owner):
    owner = owner.lower()
    dic = collection_man.find_one({'owner': owner})
    dic['main_value'] = 0
    dic['prim_value'] = 0
    for pl in collection.find({'info.current_team.owner': owner,'info.current_team.squad': 'main'}):
        dic['main_value'] += pl['info']['stats']['Qt_A']
    for pl in collection.find({'info.current_team.owner': owner,'info.current_team.squad': 'primavera'}):
        dic['prim_value'] += pl['info']['stats']['Qt_A']
    return dic

In [14]:
def count_prizes(palmares):
    sc=ch=cop=sup=ig=pv=cf=po=ca=0
    for prize in palmares:
        if prize['Type'] == 'Coppa di Lega':
            cop+=1
        elif prize['Type'] == 'Scudetto':
            sc+=1
        elif prize['Type'] == 'Champions':
            ch+=1
        elif prize['Type'] == 'Supercoppa':
            sup+=1
        elif prize['Type'] == 'Porta Violata':
            pv +=1
        elif prize['Type'] == 'Cartellino Facile':
            cf +=1
        elif prize['Type'] == 'Panchina D\'Oro':
            po +=1
        elif prize['Type'] == 'Caduti':
            ca +=1
        ig=pv+cf+po+ca
    
    return {'sc': sc, 'ch': ch, 'cop': cop, 'sup': sup, 'ig': ig,'pv':pv, 'cf':cf, 'po':po, 'ca':ca}

In [15]:
@anvil.server.callable

def man_data_tot(owner):
    
    owner = owner.lower()
    dic = collection_man.find_one({'owner': owner})
    team_name = dic['team_name']
    dic_out = {}
    
    dic_out['budget'] = dic['budget']
    dic_out['tot_wins'] = dic['total_wins']
    if len(dic['fines']):
        dic_out['tot_fines'] = pd.DataFrame(dic['fines']).Fine_eur.sum()
    else:
        dic_out['tot_fines'] = 0
    
    dic_out['tot_value'] = 0
    dic_out['tot_cost'] = 0
    for pl in collection.find({'info.contract.owner': owner}):
        dic_out['tot_value'] += pl['info']['stats']['Qt_A']
        dic_out['tot_cost'] += pl['info']['contract']['cost']
    
    dic_out['prizes'] = count_prizes(dic['palmares'])
    
    #card = anvil.media.from_file('Logos/'+owner+dic_res[res]+'.png','image/png')
    
    return dic_out, team_name

In [16]:
@anvil.server.callable
def man_data_all():
    dic_trophies = []
    dic_ig = []
    for owner in Teams.keys():
        dic_in, team_name = man_data_tot(owner)
        dic = {'team_logo_rep':owner,
            'team_name': team_name,
         'sc': dic_in['prizes']['sc'],
         'ch': dic_in['prizes']['ch'],
         'cop': dic_in['prizes']['cop'],
         'sup': dic_in['prizes']['sup'],
         'tot':dic_in['prizes']['sc']+dic_in['prizes']['ch']+dic_in['prizes']['cop']+dic_in['prizes']['sup']
         }
        dic_2 ={'team_logo_rep':owner,
            'team_name': team_name,
         'pv': dic_in['prizes']['pv'],
         'cf': dic_in['prizes']['cf'],
         'po': dic_in['prizes']['po'],
         'ca': dic_in['prizes']['ca'],
         'tot':dic_in['prizes']['pv']+dic_in['prizes']['cf']+dic_in['prizes']['po']+dic_in['prizes']['ca']
         }
        dic_trophies.append(dic)
        dic_ig.append(dic_2)
    return dic_trophies, dic_ig

In [17]:
@anvil.server.callable
#cost is yet to fix

def finance_managers_data(SC = 360, LTL = 460): #SC and LTL will be determined from previous seasons, saved on mongoDB
    table_filler_1 = []
    table_filler_2 = []
    
    for owner in Teams.keys():
        dict_out_1 = {}
        dict_out_2 = {}
        dic,  team_name = man_data_tot(owner)
        
        _, value_init_main, value_now_main, mean_age_main ,_, tot_cost_main = rose_funct(owner, 'main')
        _, value_init_prima, value_now_prima, mean_age_prima ,_, tot_cost_main = rose_funct(owner, 'primavera')
        
        dict_out_1['team'] = owner
        dict_out_2['team'] = owner
        
        dict_out_1['extra_budget'] = max(0, 100 + SC - int(value_now_main))
        dict_out_1['luxury_tax'] = max(0, int(value_now_main) - LTL)
        dict_out_1['budget'] = dic['budget']
        dict_out_1['draft_pick'] = 'n/a'
        dict_out_2['trophies'] = dic['prizes']['sc']+dic['prizes']['ch']+dic['prizes']['cop']+dic['prizes']['sup']
        dict_out_2['trophies_ig'] = dic['prizes']['ig']
        dict_out_2['tot_wins'] = dic['tot_wins']
        dict_out_2['tot_fines'] = dic['tot_fines']
        
        table_filler_1.append(dict_out_1)
        table_filler_2.append(dict_out_2)

    return table_filler_1, table_filler_2

In [18]:
@anvil.server.callable
#cost is yet to fix

def finance_one_manager_data(owner, SC = 360, LTL = 460): #SC and LTL will be determined from previous seasons, saved on mongoDB
    table_filler_1 = []
    
    owner = owner.lower()
    dic = collection_man.find_one({'owner': owner})
    dic_2,  team_name = man_data_tot(owner)
    
    _, value_init_main, value_now_main, mean_age_main ,_, tot_cost_main= rose_funct(owner, 'main')
    _, value_init_prima, value_now_prima, mean_age_prima ,_, tot_cost_prima= rose_funct(owner, 'primavera')
    
    dic['extra_budget'] = max(0, 100 + SC - int(value_now_main))
    dic['luxury_tax'] = max(0, int(value_now_main) - LTL)
    dic['draft_pick'] = 'n/a'
    dic['tot_fines'] = dic_2['tot_fines']
    dic['main_value'] = value_now_main
    dic['main_cost'] = value_now_main
    dic['prima_cost'] = value_now_main
    
    return dic
    
    

In [19]:
@anvil.server.callable
#cost is yet to fix

def rose_managers_data(SC = 360, LTL = 460):
    table_filler = []
    median_dict ={'Val_M':[],
                 'Val_P':[],
                 'Cost_M':[],
                 'Cost_P':[],
                 'Age':[]}
    for owner in Teams.keys():
        dict_out = {}
        dic,  team_name = man_data_tot(owner)
        
        _, value_init_main, value_now_main, mean_age_main ,_,tot_cost_main = rose_funct(owner, 'main')
        _, value_init_prima, value_now_prima, mean_age_prima ,_,tot_cost_prima= rose_funct(owner, 'primavera')
        
        dict_out['team'] = owner
        dict_out['extra_budget'] = max(0, 100 + SC - int(value_now_main))
        dict_out['luxury_tax'] = max(0, int(value_now_main) - LTL)
        dict_out['main_value'] = value_now_main
        dict_out['prima_value'] = value_now_prima
        dict_out['main_cost'] = tot_cost_main
        dict_out['prima_cost'] = tot_cost_main
        dict_out['mean_age'] = mean_age_main
        dict_out['draft_pick'] = 'n/a'
        dict_out['trophies'] = dic['prizes']['sc']+dic['prizes']['ch']+dic['prizes']['cop']+dic['prizes']['sup']
        dict_out['trophies_ig'] = dic['prizes']['ig']
        dict_out['tot_wins'] = dic['tot_wins']
        dict_out['tot_fines'] = dic['tot_fines']
        
        table_filler.append(dict_out)
        
        median_dict['Val_M'].append(int(value_now_main))
        median_dict['Val_P'].append(int(value_now_prima))
        median_dict['Age'].append(int(mean_age_main))
        median_dict['Cost_M'].append(int(tot_cost_main))
        median_dict['Cost_P'].append(int(tot_cost_prima))
    
    median_dict['Val_M'] = str(np.median(median_dict['Val_M']))
    median_dict['Val_P'] = str(np.median(median_dict['Val_P']))
    median_dict['Age'] = str(np.median(median_dict['Age']))
    median_dict['Cost_M'] = str(np.median(median_dict['Cost_M']))
        
    return table_filler, median_dict

In [20]:
@anvil.server.callable
def pl_names():
    posts = collection.find({})
    players = []
    for post in posts:
        players.append(post['name'])
    return sorted(players)

In [21]:
@anvil.server.callable
def pl_info(name):
    dic = collection.find_one({'name': name})
    dic_stats = dic['info']['stats']
    
    name_url = dic['name']
    name_url = name_url.replace(' ','-')
    name_url = name_url.replace('.','')
    
    card = URLMedia('https://content.fantacalcio.it/web/campioncini/card/'+name_url+'.jpg')
    stats_link = 'https://www.fantacalcio.it/squadre/'+dic['info']['personal_info']['team_real']+'/'+name_url+'/'+str(dic['_id'])
    
    return dic, dic_stats, card, stats_link

In [22]:
@anvil.server.callable
def find_trans(name = False, operation = False, year = False, owner = False):
    filt = {}
    if name:
        filt['name'] ={'$regex':''+name,'$options':'i'}
    if operation:
        filt['operation'] = operation
    if year:
        filt['date'] ={'$regex':'^'+str(year),'$options':'i'}
    posts = []
    for dic in collection_tr.find(filt):
        owner = 'N/A'
        if 'owner' in dic.keys():
            owner = dic['owner']
        elif 'new_owner' in dic.keys():
            owner = dic['new_owner']
        elif 'exch_owner' in dic.keys():
            owner = dic['exch_owner']
            
        posts.append({'name': dic['name'],'operation':dic['operation'],'date':dic['date'], 'owner': owner})
    
    return posts

In [23]:
@anvil.server.callable
def find_players(name):
    filt = {}
    filt['name'] ={'$regex':''+name,'$options':'i'}
    posts = []
    for dic in collection.find(filt):
            
        posts.append(dic['name'])
    
    return posts

In [24]:
@anvil.server.callable
def load_plot_IG(plot):
    premio_plot(Results, giornate, Teams, Logos, plot)
    card = anvil.media.from_file('Plots/plot_'+ plot +'.png','image/png')
    return card

In [25]:
def load_plot_C(plot):
    #fortuna_evo(Results, Teams, Tot_per_round)
    #C_factor_logos_2(Total, giornate, Teams, tot_giornate, Logos)
    card = anvil.media.from_file('Plots/C_fact_'+ plot +'.png','image/png')
    return card

In [26]:
@anvil.server.callable
def load_plot_C_all(regen = False):
    if regen:
        generate_plots()
    card_hist = anvil.media.from_file('Plots/C_fact_'+ 'Historic' +'.png','image/png')
    card_tot = anvil.media.from_file('Plots/C_fact_'+ 'Total' +'.png','image/png')
    return giornate, card_tot, card_hist

In [27]:
@anvil.server.callable
def load_logo(name, res = 'mid'):
    dic_res = {
        'low' : '-48',
        'mid' : '-96',
        'high' : '-250',
    }
    path = 'Logos/'+name + dic_res[res]+'.png'
    card = anvil.media.from_file(path,'image/png')
    return card

In [28]:
#fortuna_evo(Results, Teams, Tot_per_round)
#C_factor_logos_2(Total, giornate, Teams, tot_giornate, Logos)
#for premio in ['Caduti','Porta Violata','Catenaccio','Panchina Oro','Cartellino Facile']:
 #   premio_plot(Results, giornate, Teams, Logos, premio)

In [29]:
@anvil.server.callable
def load_IGNOBEL_db(plot):
    #tot = premio_plot(Results, giornate, Teams, Logos, plot)
    list_IG = sorted(dict_out[plot], key = lambda i: i['points'], reverse = True)
    i=1
    prize = 7 #to be determined via mongodb
    for el in list_IG:
        el['position'] = i
        el['team'] = el['team'].capitalize()
        el['prize'] = prize
        i+=1
    card = anvil.media.from_file('Plots/plot_'+ plot +'.png','image/png')
    #out_plot = load_plot_IG(plot)
    montepremi = str(4*prize) 
    return card, list_IG, giornate, montepremi

In [30]:
@anvil.server.callable
def load_IGNOBEL_db_all():
    montepremi = 0
    list_IG = {}
    card = {}
    for plot in ['Porta Violata', 'Cartellino Facile', 'Panchina Oro', 'Caduti']:
        list_IG[plot] = sorted(dict_out[plot], key = lambda i: i['points'], reverse = True)
        i=1
        prize = 7 #to be determined via mongodb
        for el in list_IG[plot]:
            el['position'] = i
            el['team'] = el['team'].capitalize()
            el['prize'] = prize
            i+=1
        #card[plot] = anvil.media.from_file('Plots/plot_'+ plot +'.png','image/png')
        montepremi += prize
    return list_IG, giornate, montepremi

In [31]:
@anvil.server.callable
def load_IGNOBEL_plots():
    plot_dict = {}
    generate_plots()
    for plot in ['Porta Violata', 'Cartellino Facile', 'Panchina Oro', 'Caduti']:
        plot_dict[plot] = anvil.media.from_file('Plots/plot_'+ plot +'.png','image/png')

    return plot_dict

In [32]:
@anvil.server.callable
def all_players():
    down = list(collection.find({}))
    name_list = list(pd.DataFrame(down).name)
    return name_list

In [33]:
def transfer_list(name):
    transfers = list(collection_tr.find({'name': name}))#+list(collection_tr.find({'exchange_player': name}))
    return transfers

In [34]:
@anvil.server.callable
def full_pl_info(name):
    dic = collection.find_one({'name': name})
    
    name_url = dic['name']
    name_url = name_url.replace(' ','-')
    name_url = name_url.replace('.','')
    
    card = URLMedia('https://content.fantacalcio.it/web/campioncini/card/'+name_url+'.jpg')
    
    if dic['info']['personal_info']['team_real'] is None:
        stats_link = ''
    else:
        stats_link = 'https://www.fantacalcio.it/squadre/'+dic['info']['personal_info']['team_real']+'/'+name_url+'/'+str(dic['_id'])
    
    return dic, card, stats_link, transfer_list(name)

In [35]:
a = {'name': '',
'operation': '',
'previous_owner': '',
'new_owner': '',
'exchange_player': '',
'squad': '',
'previous_squad': '',
'cost': '', 
'date': '',
'quotation_to_date': ''
          }

In [36]:
@anvil.server.callable
def save_transfer_mongo(dic, player = False, loan_info = False):
    now = datetime.today()
    Id = now.strftime("%Y%m%d%H%M%S")
    
    
    res = dic
    res['_id'] = Id
    
    if loan_info:
        date_now = date.today()
        m, y = date_now.month, date_now.year
        if m > 7:
            y = y + 1
        
        loan_info['expire_date'] = str(y)+'/07/31'
        res['loan_info'] = loan_info
    
    
        
        
    
    collection_tr.insert_one(res)
    
    if player:
        if dic['operation'] in ['Asta', 'Draft', 'Acquisto', 'Scambio']:
            collection.update_one({'name':dic['name']},{'$set':{'info.contract.start_date':dic['date']}})
            collection.update_one({'name':dic['name']},{'$set':{'info.contract.cost':dic['cost']}})
            collection.update_one({'name':dic['name']},{'$set':{'info.contract.acquisition_mode':dic['operation']}})
            collection.update_one({'name':dic['name']},{'$set':{'info.contract.previous_owner':dic['previous_owner']}})
        
        if loan_info:
            collection.update_one({'name':dic['name']},{'$set':{'info.current_team.on_loan':True}})
            collection.update_one({'name':dic['name']},{'$set':{'info.current_team.loan_info':loan_info}})
        
        collection.update_one({'name':dic['name']},{'$set':{'info.current_team.owner':dic['new_owner']}})
        collection.update_one({'name':dic['name']},{'$set':{'info.current_team.squad':dic['squad']}})
        collection.update_one({'name':dic['name']},{'$set':{'info.current_team.previous_team':dic['previous_owner']+', '+dic['previous_squad']}})
        collection.update_one({'name':dic['name']},{'$set':{'info.current_team.quotation_initial':dic['quotation_to_date']}})
    
    return res


    